In [6]:
#Importing libraries

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
from datetime import datetime

In [ ]:
# Importing directories
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../..")))

from src.utils.config import PROCESSED_DATA_STOCK_DIR

In [8]:
# Importing data 
stock_data = pd.read_csv(os.path.join(PROCESSED_DATA_STOCK_DIR, f'{datetime.now().strftime("%Y-%m-%d")}_cleaned_data.csv'))

In [9]:
# Looking at top 5 rows of the dataset
stock_data.head()

,ds,open,high,low,y,volume
0,2025-09-05 15:08:00,1243.380,1243.38,1243.330,1243.33,94
1,2025-09-05 15:07:00,1243.420,1243.51,1243.420,1243.51,292
2,2025-09-05 15:06:00,1243.320,1243.88,1243.320,1243.50,110
3,2025-09-05 15:05:00,1242.665,1242.99,1242.665,1242.88,214
4,2025-09-05 15:04:00,1242.655,1242.78,1242.610,1242.69,260


In [10]:
# Looking at bottom 5 rows of the dataset
stock_data.tail()

,ds,open,high,low,y,volume
4995,2025-08-18 14:48:00,1246.51501,1246.51501,1246.19495,1246.35535,2085
4996,2025-08-18 14:47:00,1246.52002,1246.96997,1246.00745,1246.00745,2640
4997,2025-08-18 14:46:00,1245.72998,1245.79504,1245.72998,1245.79504,1875
4998,2025-08-18 14:45:00,1246.06995,1246.08997,1245.92004,1245.92505,3204
4999,2025-08-18 14:44:00,1246.19995,1246.19995,1246.15503,1246.15503,2105
